In [1]:
import mynn
from mynn.layers.dense import dense
from mynn.optimizers.sgd import SGD
from mygrad.nnet.losses.margin_ranking_loss import margin_ranking_loss
import mygrad as mg
from noggin import create_plot
import numpy as np
import pickle
from mygrad.nnet.initializers import glorot_normal
%matplotlib notebook

In [2]:
from coco_class import Coco
from generate_triplets import generate_triplets
class Img2Caption:
    def __init__(self, dim_input: int, dim_output: int):
        """
        Initializes the Img2Caption Linear Encoder
        Parameters
        ----------
        dim_input the dimensions of the input descriptor
        dim_output the dimensions of the encoded value
        """
        self.layer = dense(dim_input, dim_output, weight_initializer=glorot_normal)
        
    def __call__(self, descriptor):
        """
        Runs the forward pass of the model
        Parameters
        ----------
        descriptor the descriptor vector to be processed

        Returns
        -------
        The shape-(dim_output) embedding

        """
        result = self.layer(descriptor)
        magnitude = mg.sqrt(mg.sum(mg.square(result), axis = 1, keepdims=True))
        print(f"Magnitude: {magnitude}")
        return result / magnitude
    
    @property
    def parameters(self):
        """
        Returns the parameters of the encoding layer
        Returns
        -------
        The parameters of the encoding layer
        """
        return self.layer.parameters

In [3]:
data = Coco()

glove loaded


UnsupportedError: Failed in nopython mode pipeline (step: inline calls to locally defined closures)
[1mThe use of yield in a closure is unsupported.
[1m
File "coco_class.py", line 202:[0m
[1m    def to_idf(self, vocab, counters):
        <source elided>
        N = len(counters)
[1m        nt = [sum(1 if t in counter else 0 for counter in counters) for t in vocab]
[0m        [1m^[0m[0m
[0m

In [ ]:
model = Img2Caption(512, 50)
lr = 1e-3
momentum = 0.9
optim = SGD(model.parameters, learning_rate = lr, momentum = momentum)


In [ ]:
plotter, figs, axes = create_plot(metrics=["loss", "accuracy"])

In [ ]:
# data = Coco()
with open("temp_cd.pkl", mode="rb") as file:
    data = pickle.load(file)


In [ ]:

# train, validation = generate_triplets(data) # List(Tuple[]), Shape (N,)
with open("train_triplets_v1.pkl", mode="rb") as file:
    train = pickle.load(file)
with open("val_triplets_v1.pkl", mode="rb") as file:
    validation = pickle.load(file)

In [32]:
train_d_good = np.zeros((len(train), 512))
for i in range(len(train)):
    train_d_good[i] = train[i][0].reshape(512)
train_w_good = np.zeros((len(train), 50))
for i in range(len(train)):
    train_w_good[i] = train[i][1].reshape(50)
train_d_bad = np.zeros((len(train), 512))
for i in range(len(train)):
    train_d_bad[i] = train[i][2].reshape(512)
    
val_d_good = np.zeros((len(validation), 512))
for i in range(len(validation)):
    val_d_good[i] = validation[i][0].reshape(512)
val_w_good = np.zeros((len(validation), 50))
for i in range(len(validation)):
    val_w_good[i] = validation[i][1].reshape(50)
val_d_bad = np.zeros((len(validation), 512))
for i in range(len(validation)):
    val_d_bad[i] = validation[i][2].reshape(512)

train = (train_d_good[::2], train_w_good[::2], train_d_bad[::2])
validation = (val_d_good[::2], val_w_good[::2], val_d_bad[::2])

In [8]:
d_img = train[0]
print(len(d_img))

330415


In [48]:
idx = np.arange(len(train[0]))
b = idx[:32]
bd_img = d_img[b]
bw_good = w_good[b]
bd_bad = d_bad[b]

w_bad = model(bd_bad) # Shape (32, 50)
w_img = model(bd_img)
dot_good = mg.sum(w_img * bw_good, axis=1) # Shape (32,)
dot_bad = mg.sum(bw_good * w_bad, axis=1) # Shape (32,)

loss = margin_ranking_loss(dot_good, dot_bad, 1, margin=0.1)


print(w_bad)
print(w_img)
print(dot_good.max())
print(dot_good.min())
print(dot_bad.max())
print(dot_bad.min())

loss.backward()
print(model.layer.weight.grad)

Tensor([[ 0.07102041,  0.04983361,  0.0343528 , ...,  0.19443091,
          0.03889689, -0.09427323],
        [-0.03451658, -0.10148807, -0.12066847, ...,  0.1108868 ,
         -0.08822447,  0.15261918],
        [ 0.06754778,  0.09816844, -0.04274904, ...,  0.28116812,
          0.03204005,  0.13971078],
        ...,
        [ 0.08709974,  0.03563017, -0.05917015, ...,  0.00778787,
          0.05407839,  0.02655266],
        [ 0.05693169,  0.09842793,  0.27056281, ..., -0.07260665,
          0.10194122,  0.14205077],
        [-0.1219042 ,  0.03351373,  0.10670598, ...,  0.08469758,
          0.0314163 ,  0.14988822]])
Tensor([[ 0.10788981, -0.05371741,  0.08718307, ...,  0.22560639,
          0.09745083,  0.34326418],
        [ 0.10788981, -0.05371741,  0.08718307, ...,  0.22560639,
          0.09745083,  0.34326418],
        [ 0.10788981, -0.05371741,  0.08718307, ...,  0.22560639,
          0.09745083,  0.34326418],
        ...,
        [ 0.1145021 ,  0.02416583,  0.03840153, ..., -0

In [68]:
epochs = 5
batch_size = 32

d_img = train[0]
w_good = train[1]
w_good = w_good / np.linalg.norm(w_good, axis=1, keepdims=True)
d_bad = train[2]
for ep in range(epochs):
    idx = np.arange(len(train[0]))
    np.random.shuffle(idx)
    for batch_cnt in range(len(train[0]) // batch_size + 1):
        b = idx[batch_cnt * batch_size:(batch_cnt + 1) * batch_size]
        
        bd_img = d_img[b]
        bw_good = w_good[b]
        bd_bad = d_bad[b]
        
        w_bad = model(bd_bad) # Shape (32, 50)
        w_img = model(bd_img) # Shape (32, 50)
        
        dot_good = mg.sum(w_img * bw_good, axis=1) # Shape (32,)
        print(dot_good.mean())
            
        dot_bad = mg.sum(bw_good * w_bad, axis=1) # Shape (32,)
        print(dot_bad.mean())
        acc = np.mean(dot_good > dot_bad)
        
        loss = margin_ranking_loss(dot_good, dot_bad, 1, margin=0.1)

#         loss.backward()

#         optim.step()

#         loss.null_gradients()

        plotter.set_train_batch(metrics={"loss": loss.item(), "accuracy":acc}, batch_size=batch_size)

    plotter.set_train_epoch()


Magnitude: Tensor([[ 9.41932989],
        [14.05186485],
        [15.00650524],
        [13.96475659],
        [10.87193519],
        [11.23819091],
        [12.12576711],
        [14.620452  ],
        [11.95419195],
        [12.43597413],
        [13.60161625],
        [10.94133258],
        [13.99833998],
        [10.20221382],
        [12.4920618 ],
        [14.16406901],
        [10.50251005],
        [11.6349799 ],
        [11.70527585],
        [12.11070562],
        [13.16509561],
        [10.6547747 ],
        [13.86550942],
        [ 9.1911298 ],
        [13.08523392],
        [17.25232339],
        [11.52983391],
        [13.92606404],
        [11.30884783],
        [12.6674169 ],
        [16.10357274],
        [10.39388921]])
Magnitude: Tensor([[15.5219487 ],
        [15.84724731],
        [11.92386867],
        [10.17013627],
        [14.55503955],
        [14.04881801],
        [14.68165111],
        [11.39025454],
        [12.28931301],
        [ 9.98029949],
        [12

        [11.42369339]])
Magnitude: Tensor([[10.29800529],
        [11.04267398],
        [13.42107229],
        [11.91699105],
        [13.17943801],
        [14.87548186],
        [10.6766481 ],
        [12.30645157],
        [10.55042199],
        [12.3647517 ],
        [13.67247027],
        [14.2242488 ],
        [10.41762397],
        [13.64283672],
        [13.95809194],
        [11.98186505],
        [13.18236541],
        [10.31333988],
        [12.34982823],
        [11.67667743],
        [ 9.71310886],
        [11.29765447],
        [10.46090148],
        [11.56447483],
        [11.07153615],
        [13.34421882],
        [12.57609484],
        [12.99515012],
        [12.17072816],
        [ 9.71964391],
        [11.44518983],
        [11.01651885]])
Tensor(nan)
Tensor(nan)
Magnitude: Tensor([[ 9.16966855],
        [11.90934898],
        [11.37919327],
        [12.53444679],
        [12.51066698],
        [11.4246511 ],
        [13.22395775],
        [12.15744071],
        [

        [11.87987526]])
Tensor(nan)
Tensor(nan)
Magnitude: Tensor([[13.42234896],
        [ 8.63618006],
        [14.78653499],
        [11.10102329],
        [14.36984339],
        [10.20872081],
        [11.90458504],
        [11.63171907],
        [13.61379067],
        [10.44469422],
        [13.08234822],
        [11.28475709],
        [11.56624979],
        [11.86941495],
        [13.69309812],
        [10.30477612],
        [11.84480524],
        [16.30574347],
        [12.87806285],
        [11.52397114],
        [11.49255661],
        [11.84938569],
        [11.36841451],
        [12.09462755],
        [11.84566234],
        [11.88893113],
        [15.61998591],
        [13.19391588],
        [ 9.66875983],
        [11.29625851],
        [11.6311987 ],
        [13.14918294]])
Magnitude: Tensor([[11.98455992],
        [11.35707492],
        [13.89703641],
        [12.54291298],
        [13.78763454],
        [10.88045389],
        [10.31282073],
        [15.5242554 ],
        [

        [10.44009415]])
Magnitude: Tensor([[11.59821234],
        [14.70191613],
        [15.18207873],
        [13.37980072],
        [13.64998099],
        [12.12886019],
        [12.19582961],
        [13.51829405],
        [12.86185432],
        [10.52619899],
        [12.4107139 ],
        [13.50327211],
        [12.7692449 ],
        [11.65499897],
        [11.50975243],
        [11.33279257],
        [14.76593287],
        [14.51429314],
        [13.98934656],
        [11.79214838],
        [14.59007329],
        [12.55768918],
        [15.32875842],
        [14.3424996 ],
        [ 9.2753249 ],
        [10.59382029],
        [13.57106221],
        [11.85657123],
        [14.03823684],
        [ 9.16413082],
        [15.43876568],
        [ 9.57167652]])
Tensor(nan)
Tensor(nan)
Magnitude: Tensor([[10.04402993],
        [16.99460905],
        [14.54497871],
        [13.37828084],
        [ 9.89164314],
        [15.78658596],
        [12.96683117],
        [11.85504823],
        [

        [11.16036844]])
Magnitude: Tensor([[12.66674025],
        [11.21930095],
        [10.3029866 ],
        [14.00786075],
        [12.66997351],
        [10.28631914],
        [13.10331625],
        [14.29934654],
        [15.67153557],
        [10.15183701],
        [12.83581993],
        [11.51924588],
        [10.88088265],
        [11.23441   ],
        [11.16469594],
        [13.56892157],
        [10.90224248],
        [12.72182979],
        [12.96741812],
        [14.42030774],
        [14.47407961],
        [15.63405084],
        [12.94538389],
        [12.63393709],
        [14.01927888],
        [11.59433799],
        [11.1290627 ],
        [15.03788986],
        [12.62773279],
        [13.6958252 ],
        [15.6986829 ],
        [14.17139478]])
Tensor(-0.10413073)
Tensor(-0.12446104)
Magnitude: Tensor([[13.41548963],
        [11.26200795],
        [11.4113053 ],
        [13.40430157],
        [11.89755153],
        [13.33158947],
        [ 8.91310597],
        [13.6825

...

Magnitude: Tensor([[10.62192247],
        [14.41134186],
        [11.24799804],
        [14.54979418],
        [12.53750018],
        [12.19217045],
        [11.52893626],
        [12.1286169 ],
        [13.0092268 ],
        [12.68745578],
        [11.63318807],
        [14.63076837],
        [10.82225853],
        [12.53671731],
        [12.93639879],
        [10.44765098],
        [11.26972809],
        [11.04303803],
        [10.25131166],
        [12.56552363],
        [14.34164725],
        [16.66390748],
        [11.58261775],
        [12.17281289],
        [12.71398474],
        [11.95219471],
        [11.91512978],
        [10.14574726],
        [10.1658418 ],
        [12.98688606],
        [12.26981373],
        [11.54684487]])
Tensor(nan)
Tensor(nan)
Magnitude: Tensor([[10.80721465],
        [11.62554423],
        [12.62770341],
        [11.11846124],
        [12.63203194],
        [12.30672093],
        [ 9.97469295],
        [11.30974379],
        [12.94524535],
        [1

        [10.90899627]])
Magnitude: Tensor([[14.17752264],
        [11.21763742],
        [20.58108979],
        [15.51870441],
        [12.55539381],
        [10.13394439],
        [13.71667854],
        [13.3068207 ],
        [13.25607377],
        [14.06245306],
        [12.85381833],
        [16.67475112],
        [16.00476263],
        [13.67216465],
        [ 9.60888057],
        [ 9.41336651],
        [11.07941224],
        [14.70007041],
        [10.2653674 ],
        [12.24569805],
        [13.27800615],
        [13.06595531],
        [12.85344538],
        [ 8.29105744],
        [12.20952554],
        [15.18720396],
        [13.04199521],
        [12.5622774 ],
        [10.96802598],
        [14.23578652],
        [13.10696665],
        [12.15689115]])
Tensor(nan)
Tensor(nan)
Magnitude: Tensor([[13.62868787],
        [12.87743081],
        [13.66019758],
        [18.81215673],
        [12.74735792],
        [16.13029072],
        [11.13091066],
        [11.81212138],
        [

Tensor(-0.11698012)
Tensor(-0.09341443)
Magnitude: Tensor([[12.13916409],
        [11.5023182 ],
        [11.25200263],
        [10.63271113],
        [11.89759726],
        [11.06653282],
        [15.67349469],
        [ 9.72318463],
        [13.95690791],
        [14.16691876],
        [12.05794412],
        [11.53463943],
        [11.48398997],
        [10.97643251],
        [11.62078864],
        [13.75971377],
        [11.51899405],
        [14.826031  ],
        [13.15409202],
        [13.29667069],
        [11.23728227],
        [13.30828605],
        [11.29625851],
        [12.26577947],
        [12.31066383],
        [14.20370389],
        [10.08218263],
        [11.53516257],
        [12.4899927 ],
        [13.3874209 ],
        [16.61616017],
        [13.46305829]])
Magnitude: Tensor([[ 9.62524637],
        [10.77530671],
        [11.82340692],
        [13.20044195],
        [ 9.70038053],
        [13.51564351],
        [10.98315711],
        [12.3475146 ],
        [11.99089

Magnitude: Tensor([[13.18946487],
        [12.54181678],
        [11.39882788],
        [ 9.60795393],
        [14.67832477],
        [10.90352761],
        [13.3711116 ],
        [14.1676766 ],
        [13.01420544],
        [13.14936006],
        [11.02458955],
        [12.26625136],
        [12.43966865],
        [11.54962786],
        [10.88459106],
        [11.86484727],
        [11.84360664],
        [13.93373964],
        [13.83531472],
        [13.42264634],
        [15.3904627 ],
        [13.58477129],
        [14.1193625 ],
        [12.38436619],
        [10.52929894],
        [11.35573922],
        [11.32656158],
        [12.31754398],
        [12.19554042],
        [15.82087342],
        [ 9.99640479],
        [15.1245522 ]])
Magnitude: Tensor([[ 7.91378148],
        [10.84494007],
        [12.50393397],
        [14.05405378],
        [11.69046706],
        [10.16170263],
        [12.43244024],
        [11.74462295],
        [10.32028962],
        [11.85441488],
        [13

KeyboardInterrupt: 

In [61]:
# print(dot_good)
# print(dot_bad)
# print(w_bad)
# print(w_img)
print(bd_img)
print(np.linalg.norm(bw_good,axis=1))
print(bd_bad)

[[1.38372242 0.79107618 2.47902846 ... 1.77728534 1.84506273 1.9262557 ]
 [1.95013368 2.13824582 0.75338435 ... 0.06614064 1.81178069 0.76349992]
 [0.34629354 0.23912963 0.28421357 ... 0.67701119 0.35397545 2.34805465]
 ...
 [1.00953126 2.89725542 0.42719081 ... 0.57562923 0.09881391 0.05302662]
 [0.87503332 1.29086626 0.1983432  ... 0.42752948 0.60266405 0.44355509]
 [0.         1.32049751 0.02357272 ... 1.12617528 0.06949332 0.41221076]]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
[[0.28393805 0.62524104 0.14700276 ... 0.19647674 4.06395721 0.51066828]
 [0.45613325 1.80200303 0.6274848  ... 0.58377868 0.69437206 0.31062904]
 [1.30805516 0.74788821 1.31198823 ... 0.78026295 0.31707814 0.82756519]
 ...
 [0.29385543 0.01552426 0.2205736  ... 0.65662795 0.82849151 0.21491462]
 [1.55326176 0.61116642 3.88131714 ... 0.78870904 2.28962803 1.04799771]
 [0.74403208 1.24362862 0.10904855 ... 0.74419969 1.30684519 0.28500128]]
